# Keras Model for Casava Leaf Disease Classification

### Importing Required Library for the classification

In [ ]:
import numpy as np 
import pandas as pd
#import gc
#import sklearn
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from tqdm import tqdm
from keras.preprocessing import image
from keras.preprocessing.image import img_to_array, load_img, ImageDataGenerator
import cv2
from random import shuffle
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense, Activation, BatchNormalization


data = pd.read_csv('../input/cassava-leaf-disease-classification/train.csv')
###Converting Image label as string from integer.


### I decided to cut down some training Images also their was imbalance of dataset as code '3' is in majority, thus, adjusted random sampling 

### Also used 'del' & 'gc.collect()' free up some RAM memory.

In [ ]:
data['label'] = data['label'].astype('string')

frame  =[
        data[data['label']=='3'].sample(n=900,replace=True),
        data[data['label']=='1'].sample(n=900,replace=True),
        data[data['label']=='2'].sample(n=900,replace=True),
        data[data['label']=='4'].sample(n=900,replace=True),
        data[data['label']=='0'].sample(n=900,replace=True)]


df = pd.concat(frame)

## Image Processing

In [ ]:

images_train = []
label_image = []

for (file,label) in tqdm(zip(df['image_id'],df['label'])):
    img = cv2.imread('../input/cassava-leaf-disease-classification/train_images/'+file, cv2.IMREAD_COLOR)       
    img = cv2.resize(img, (100,100))
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    imgs = img.reshape((1,) + img.shape)
    img = image.img_to_array(img)
    data_generator = ImageDataGenerator(rotation_range=45, 
                                        brightness_range=(0.5, 1.2),
                                        width_shift_range=0.2,
                                        height_shift_range=0.2,
                                        shear_range=10,
                                        horizontal_flip=True,
                                        vertical_flip=True,
                                        zoom_range=[.1, .8])
    data_generator.fit(imgs)
    image_iterator = data_generator.flow(imgs)
    for x in range(3):
        img_transformed=image_iterator.next()[0].astype('int')/255
        images_train.append(img_transformed)
        label_image.append(label)

In [ ]:
train = np.array(images_train)

### Splitting the data into train and test

In [ ]:

xtrain, xtest, ytrain, ytest = train_test_split(train,label_image, test_size=0.2, random_state = 42 )



### Categorisation

In [ ]:
Y = ytrain
Ytrain = to_categorical(Y)

z = ytest
Ytest = to_categorical(z)

#del ytrain,ytest
#gc.collect()

### Image Processing Submission test file

In [ ]:
from keras.preprocessing import image

img = cv2.imread('../input/cassava-leaf-disease-classification/test_images/2216849948.jpg', cv2.IMREAD_COLOR)
img = cv2.resize(img, (100, 100))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = image.img_to_array(img)
img = img/255.0
Test = np.array(img)
Test = Test.reshape(1,Test.shape[0],Test.shape[1],Test.shape[2])

## Building the Model

In [ ]:
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512, activation='relu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))
model.add(Dense(5, activation='sigmoid'))


model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['accuracy'])

model.summary()

In [ ]:
model.fit(xtrain, Ytrain,epochs=40, batch_size = 40, validation_data=(xtest, Ytest))

In [ ]:
pred = model.predict(Test)
label_predicted=np.argmax(pred, axis=1)

label_predicted


In [ ]:
submission = pd.DataFrame({'image_id':'2216849948.jpg' ,'label':label_predicted})
submission.to_csv("submission.csv", index = False, header = True)